In [2]:
import pandas as pd
import numpy as np
import os
import time
import random
import gensim
import numpy as np
import os
import torch
from gensim.models import KeyedVectors
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

/home/fatma/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_mean_vector(model, words):
    # remove out-of-vocabulary words
    # words: target list words
    # returns average vector for all words in target list
    import numpy as np
    words = [word for word in words if word in model.index_to_key]
    if len(words) >= 1:
        return np.mean(model[words], axis=0)
    else:
        return []

def get_similarity_Scores(v1,v2):
    from numpy import dot
    from numpy.linalg import norm
    return dot(v1,v2)/(norm(v1)*norm(v2))

def normalize_data(a):
    normalizded_a = []
    amin, amax = min(a), max(a)
    for i in a:
        normalizded_a.append((i-amin) / (amax-amin))
    return normalizded_a
    
def get_similarity_scores_between_target_words_and_attribute_words(model, attribute_words, target_words):
    
    similarity_scores_summary = {}
    words = []
    mean_sim_socres = []

    target_words = [word for word in target_words if word in model.index_to_key]
    attribute_words = [word for word in attribute_words if word in model.index_to_key]
    
    if len(target_words) >= 1 and len(attribute_words) >=1:
        for i in target_words:
            similarity_scores = []
            words.append(i)
            for j in attribute_words:
                cos_sim = get_similarity_Scores(model[i], model[j])
                similarity_scores.append(cos_sim)
            mean_sim_socres.append(np.mean(normalize_data(similarity_scores)))

    similarity_scores_summary["word"] = words
    similarity_scores_summary["mean_sim_score"] = mean_sim_socres
    return similarity_scores_summary

def get_similarity_scores_between_avergare_target_vector_and_attribute_words(model, att_words, target_vector):
    # remove out-of-vocabulary words
    # att_words: wors in the attribute list
    # target_vector: average vector of all words in target group
    similarity_scores_summary = {}
    words_list = []
    similarity_scores = []
    words = [word for word in att_words if word in model.index_to_key]
    print(len(words))
    if len(words) >= 1:
        for i in words:
            print(i)
            words_list.append(i)
            cos_sim = get_similarity_Scores(model[i], target_vector)
            similarity_scores.append(cos_sim)
    similarity_scores_summary["words"] = words_list
    similarity_scores_summary["cos_sim_scores"] = similarity_scores
    similarity_scores_summary["sim_score"] = normalize_data(similarity_scores)

    return similarity_scores_summary


## Read profane words

In [3]:
data_folder = "../../Data/"
embeddings_folder = data_folder+"/word embeddings/"
results_folder = "Results/ethnicity_similarities/word2vec/"

In [4]:
profane_words = open(data_folder+"swear_words.txt").read().splitlines()

## read word embeddings

In [5]:
def get_google_news_embeddings(filename):
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=True)
    return w2v_model


## word similarity

In [6]:
# convert text to w2v
#from gensim.scripts.glove2word2vec import glove2word2vec
#GLOVE_DIR = data_folder+"/glove.6B.300d_WP.txt"
#w2v_file = data_folder+"/wv_2_glove.6B.300d_WP.txt"
#glove2word2vec(GLOVE_DIR, w2v_file)

In [7]:
w2v_model = get_google_news_embeddings(embeddings_folder+"/W2V/GoogleNews-vectors-negative300.bin")

# similair words to NOI words

In [13]:
w2v_model.most_similar("girl")

[('boy', 0.8543271422386169),
 ('teenage_girl', 0.7927975654602051),
 ('woman', 0.7494640946388245),
 ('teenager', 0.7172499299049377),
 ('schoolgirl', 0.7075953483581543),
 ('teenaged_girl', 0.6650916337966919),
 ('daughter', 0.6489864587783813),
 ('mother', 0.6478164196014404),
 ('toddler', 0.6473966836929321),
 ('girls', 0.6154742240905762)]

In [8]:
w2v_model.most_similar("queer")

[('genderqueer', 0.7026612162590027),
 ('LGBTQ', 0.6990917325019836),
 ('gay', 0.6972684860229492),
 ('LGBT', 0.6857556104660034),
 ('lesbian', 0.6770454049110413),
 ('GLBT', 0.6749699711799622),
 ('Queer', 0.6673474907875061),
 ('GLBTQ', 0.6604336500167847),
 ('queers', 0.6523240208625793),
 ('gay_bisexual_transgendered', 0.6437941193580627)]

In [9]:
w2v_model.most_similar("queers")

[('homos', 0.6862955093383789),
 ('homosexuals', 0.6555311679840088),
 ('queer', 0.6523239612579346),
 ('lesbians', 0.6243589520454407),
 ('homophobes', 0.6139503717422485),
 ('gay', 0.6086617708206177),
 ('bi_sexuals', 0.608240008354187),
 ('gays', 0.5921681523323059),
 ('LGBTs', 0.5905815362930298),
 ('GLBTs', 0.5848169922828674)]

In [18]:
w2v_model.most_similar("immigrant")

[('immigrants', 0.7985076308250427),
 ('Immigrant', 0.6984704732894897),
 ('migrant', 0.6784891486167908),
 ('illegal_immigrant', 0.6712934970855713),
 ('immigration', 0.601755678653717),
 ('Immigrants', 0.6007307171821594),
 ('Latino_immigrants', 0.5922547578811646),
 ('undocumented_immigrant', 0.5837289094924927),
 ('illegal_alien', 0.5803019404411316),
 ('emigrant', 0.5761232376098633)]

## minority and majority words and profanity vector

In [8]:
harmless_majority_words_clean = ["hetrosexual", "cisgender", "man","male","boy","son","father","husband","brother","white","caucasian",
                                 "european american","european", "norwegian", "canadian", "german", "australian", "english",
                                "french", "american", "swedish", "dutch"]

target_words = ["lesbian","gay","bisexual","transgender","tran","queer",
                "lgbt","lgbtq","homosexual", "woman", "female", "girl","wife","sister","daughter","mother", "non-binary",
               "african", "african american", "asian", "black", "hispanic", "latin", "mexican", "indian", "middle eastern",
                "arab"]

words = harmless_majority_words_clean + target_words

In [9]:
profane_vector = get_mean_vector(w2v_model, profane_words)
vec_majority_sim_dict = get_similarity_scores_between_avergare_target_vector_and_attribute_words(w2v_model, words, profane_vector)
pd.DataFrame(vec_majority_sim_dict).to_csv(results_folder+"_minority_and_majority_words_sim_to_profane_vector.csv")

43
hetrosexual
man
male
boy
son
father
husband
brother
white
caucasian
european
norwegian
canadian
german
australian
english
french
american
swedish
dutch
lesbian
gay
bisexual
transgender
tran
queer
lgbt
homosexual
woman
female
girl
wife
sister
daughter
mother
african
asian
black
hispanic
latin
mexican
indian
arab


In [ ]:
figure(figsize=(25, 10), dpi=1200)
sorted_index = np.argsort(vec_majority_sim_dict["sim_score"])

x_axis = np.arange(len(vec_majority_sim_dict["words"]))
y_axis = [vec_majority_sim_dict["sim_score"][i] for i in sorted_index]
x_ticks = [vec_majority_sim_dict["words"][i] for i in sorted_index]
plt.bar(x_axis, y_axis)
plt.xticks(x_axis, x_ticks, fontsize=40, fontweight='bold', rotation=90)
plt.ylabel("similarity score", fontsize=40, fontweight='bold')
plt.yticks(np.arange(0.1,1.1,0.1),fontsize=40, fontweight='bold')
plt.savefig(results_folder+"similarity_score_of_minority_and_majority_words_to_profanity_vector.pdf", bbox_inches='tight')